# 钉钉告警与自动修复

In [1]:
import json
import requests
from netmiko import ConnectHandler

In [3]:
device_info = {
    "device_type": "cisco_ios",
    "ip": "192.168.1.60",
    "port": 22,
    "username": "ciscouser",
    "password": "cisco@123",
    "secret" : "cisco123",
    "verbose": True
}

# 定义钉钉机器人的Webhook函数
* [钉钉机器人文档](https://developers.dingtalk.com/document/robots/custom-robot-access?spm=ding_open_doc.document.0.0.4de57b4bZRuoE6#topic-2026027)

In [4]:
def dingrobot(webhook, msg):
    headers = {"Content-Type":"application/json"}
    data = {"msgtype": "markdown",
            "markdown": {"title":"DevNet告警", "text": "【告警】\n" + msg}
           }
    response = requests.post(webhook,
                             data=json.dumps(data),
                             headers=headers)
    if response.ok:
        print("告警成功")
    else:
        print(response.text)
        
def auto_fix(cmd, webhook):
    server = ConnectHandler(**device_info)
    server.enable()
    while True:
        if_data = server.send_command(cmd)
        for one_if in if_data.split("\n"):
            if "down" in one_if:
                if_name = one_if.split()[0]
                warning_msg = "接口 {} DOWN了！".format(if_name)
                dingrobot(webhook, warning_msg)
                # 尝试自动修复
                _fix_cmd = ["interface {}".format(if_name), "no shut"]
                server.send_config_set(_fix_cmd)
                warning_msg = "接口 {} 修复了！".format(if_name)
                dingrobot(webhook, warning_msg)

# 测试效果

In [5]:
monitor_cmd = "show ip int brie"
webhook = "https://oapi.dingtalk.com/robot/send?access_token=a54e2f2ee022e9916cb2982054e4b9b726f1464cda321eb1e3fa63a881bc90ac"

In [ ]:
auto_fix(monitor_cmd, webhook)

SSH connection established to 192.168.1.60:22
Interactive SSH session established
告警成功
告警成功
告警成功
告警成功
告警成功
告警成功
告警成功
告警成功
告警成功
告警成功
